In [1]:
import argparse

from enum import Enum

import cv2
import numpy as np
import torch

from vedacore.misc import Config, load_weights
from vedadet.models import build_detector
from volksdep.converters import torch2onnx, torch2trt
from flexinfer.misc import Config, set_device
from flexinfer.preprocess import build_preprocess
from flexinfer.model import build_model
from flexinfer.postprocess import build_postprocess

In [2]:
config_path = 'configs/trainval/tinaface/tinaface_r50_fpn_bn.py'
cfg = Config.fromfile(config_path)
device = torch.cuda.current_device()
dummy_input_shape = '3,875,1280'
checkpoint = 'tinaface_r50_fpn_bn.pth'
dynamic_shape = True
opset_version = 12
out = 'tinaface_r50_fpn_bn.onnx'
verbose = False
do_constant_folding = False
model = build_detector(cfg.model)
load_weights(model, checkpoint)
model.to(device)
model.forward = model.forward_impl
shape = map(int, dummy_input_shape.split(','))
dummy_input = torch.randn(1, *shape)

In [6]:
config_path = 'tinaface.py'
cfg = Config.fromfile(config_path)
preprocess = build_preprocess(cfg.preprocess)
postprocess = build_postprocess(cfg.postprocess)

In [4]:
image_name = 'images/1.hd.jpeg'
img = cv2.imread(image_name)
data = [dict(img=img)]

cfg = Config.fromfile('tinaface.py')
image_name = 'images/1.hd.jpeg'

data = preprocess(data)
data['out'] = model(data.pop('img'))

In [7]:
result = postprocess(data)[0]

3
3


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
trt_model = torch2trt(
    model,
    dummy_input,
    log_level='ERROR',
    max_batch_size=1,
    min_input_shapes=[(3, 128, 128)],
    max_input_shapes=[(3, 1664, 1664)],
    max_workspace_size=1,
    fp16_mode=True,
    strict_type_constraints=False,
    int8_mode=False,
    int8_calibrator=None,
    opset_version=opset_version,
    do_constant_folding=do_constant_folding,
    verbose=verbose
)